In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os 
os.chdir('/content/gdrive/MyDrive/Project')

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

In [ ]:
Total_pKa=pd.read_csv('pKa.csv')
Total_hfus=pd.read_csv('hfus.csv')

In [ ]:
Y_pka=pd.read_csv('pKa.csv').iloc[:,1]
Y_hfus=pd.read_csv('hfus.csv').iloc[:,1]

In [ ]:
X_pka=pd.read_csv('pKa.csv').iloc[:,2:426]
X_hfus=pd.read_csv('hfus.csv').iloc[:,2:426]

In [ ]:
Smiles=pd.read_csv('pKa.csv').iloc[:,0]

In [ ]:
X_pka_train, X_pka_tmp, y_pka_train, y_pka_tmp = train_test_split(X_pka,Y_pka, test_size=0.2)
X_pka_valid, X_pka_test, y_pka_valid, y_pka_test = train_test_split(X_pka_tmp,y_pka_tmp, test_size=0.5)
X_hfus_train, X_hfus_tmp, y_hfus_train, y_hfus_tmp = train_test_split(X_hfus,Y_hfus, test_size=0.2)
X_hfus_valid, X_hfus_test, y_hfus_valid, y_hfus_test = train_test_split(X_hfus_tmp,y_hfus_tmp, test_size=0.5)

**Linear Regression**

In [ ]:
Linregr_pka = linear_model.ElasticNet(alpha=0.01,random_state=400,l1_ratio=0.1)
Linregr_pka.fit(X_pka_train, y_pka_train)
Linregr_hfus = linear_model.ElasticNet(alpha=0.001,random_state=400,l1_ratio=0.1)
Linregr_hfus.fit(X_hfus_train, y_hfus_train)

ElasticNet(alpha=0.001, l1_ratio=0.1, random_state=400)

In [ ]:
y_Linpred_valid_pka = Linregr_pka.predict(X_pka_valid)
print('Coefficient of determination for validation case for pKa: {}'.format(r2_score(y_pka_valid,y_Linpred_valid_pka)))
y_Linpred_valid_hfus = Linregr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case for Heat of fusion: {}'.format(r2_score(y_hfus_valid,y_Linpred_valid_hfus)))

Coefficient of determination for validation case for pKa: 0.21579485636386997
Coefficient of determination for validation case for Heat of fusion: 0.4933624960838602


In [ ]:
y_Linpred_test_pka = Linregr_pka.predict(X_pka_test)
print('Coefficient of determination for test case for pKa: {}'.format(r2_score(y_pka_test,y_Linpred_test_pka)))
y_Linpred_test_hfus = Linregr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case for Heat of fusion: {}'.format(r2_score(y_hfus_test,y_Linpred_test_hfus)))

Coefficient of determination for test case for pKa: 0.2481709207312216
Coefficient of determination for test case for Heat of fusion: 0.8897337465683952


**Polynomial Regression**

In [ ]:
pf_pka=PolynomialFeatures(degree=2)
pka_new=pf_pka.fit_transform(X_pka_train)
pf_hfus=PolynomialFeatures(degree=2)
hfus_new=pf_hfus.fit_transform(X_hfus_train)

In [ ]:
polyregr_pka = linear_model.ElasticNet(alpha=0.1,random_state=300,l1_ratio=0.1,max_iter=1000)
polyregr_pka.fit(pka_new, y_pka_train)
polyregr_hfus = linear_model.ElasticNet(alpha=0.3,random_state=300,l1_ratio=0.1,max_iter=1000)
polyregr_hfus.fit(hfus_new, y_hfus_train)

ElasticNet(alpha=0.3, l1_ratio=0.1, random_state=300)

In [ ]:
y_polypred_valid_pka = polyregr_pka.predict(pf_pka.transform(X_pka_valid))
y_polypred_valid_hfus = polyregr_hfus.predict(pf_hfus.transform(X_hfus_valid))
print('Coefficient of determination for validation case for pKa: {}'.format(r2_score(y_pka_valid,y_polypred_valid_pka)))
print('Coefficient of determination for validation case for Heat of fusion: {}'.format(r2_score(y_hfus_valid,y_polypred_valid_hfus)))

Coefficient of determination for validation case for pKa: 0.05055952943652853
Coefficient of determination for validation case for Heat of fusion: 0.6246916874552519


In [ ]:
y_polypred_test_pka = polyregr_pka.predict(pf_pka.transform(X_pka_test))
y_polypred_test_hfus = polyregr_hfus.predict(pf_hfus.transform(X_hfus_test))
print('Coefficient of determination for test case for pKa: {}'.format(r2_score(y_pka_test,y_polypred_test_pka)))
print('Coefficient of determination for test case for Heat of fusion: {}'.format(r2_score(y_hfus_test,y_polypred_test_hfus)))

Coefficient of determination for test case for pKa: 0.2645463994874475
Coefficient of determination for test case for Heat of fusion: 0.8637619946668662


**Support Vector Regression**

**Linear kernel**

In [ ]:
svr_pka_linear = SVR(kernel='linear',C=1.842,epsilon=0.002)
svr_pka_linear.fit(X_pka_train, y_pka_train)

svr_hfus_linear = SVR(kernel='linear',C=1.389,epsilon=0.002)
svr_hfus_linear.fit(X_hfus_train, y_hfus_train)

SVR(C=1.389, epsilon=0.002, kernel='linear')

In [ ]:
y_pred_linear_pka_valid = svr_pka_linear.predict(X_pka_valid)
y_pred_linear_hfus_valid = svr_hfus_linear.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_linear_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_linear_hfus_valid)))

Coefficient of determination for validation case: -0.10767230026640306
Coefficient of determination for validation case: 0.6153551999091745


In [ ]:
y_pred_linear_pka_test = svr_pka_linear.predict(X_pka_test)
y_pred_linear_hfus_test = svr_hfus_linear.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_linear_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_linear_hfus_test)))

Coefficient of determination for test case: 0.2871766320703011
Coefficient of determination for test case: 0.9056301314614107


**Polynomial kernel**

In [ ]:
svr_pka_poly = SVR(kernel='poly',degree=2,C=0.4,epsilon=0.02)
svr_pka_poly.fit(X_pka_train, y_pka_train)

svr_hfus_poly = SVR(kernel='poly',degree=2,C=100,epsilon=0.05)
svr_hfus_poly.fit(X_hfus_train, y_hfus_train)

SVR(C=100, degree=2, epsilon=0.05, kernel='poly')

In [ ]:
y_pred_poly_pka_valid = svr_pka_poly.predict(X_pka_valid)
y_pred_poly_hfus_valid = svr_hfus_poly.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_poly_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_poly_hfus_valid)))

Coefficient of determination for validation case: 0.18818897827314085
Coefficient of determination for validation case: 0.5713270529948142


In [ ]:
y_pred_poly_pka_test = svr_pka_poly.predict(X_pka_test)
y_pred_poly_hfus_test = svr_hfus_poly.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_poly_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_poly_hfus_test)))

Coefficient of determination for test case: 0.1621354675280553
Coefficient of determination for test case: 0.8695656198101545


**Rbf Kernel**

In [ ]:
svr_pka = SVR(kernel='rbf',C=60,epsilon=0.01)
svr_pka.fit(X_pka_train, y_pka_train)

svr_hfus = SVR(kernel='rbf',C=60,epsilon=0.01)
svr_hfus.fit(X_hfus_train, y_hfus_train)

SVR(C=60, epsilon=0.01)

In [ ]:
y_pred_rbf_pka_valid = svr_pka.predict(X_pka_valid)
y_pred_rbf_hfus_valid = svr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_rbf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_rbf_hfus_valid)))

Coefficient of determination for validation case: 0.3742975048832139
Coefficient of determination for validation case: 0.6926307862528139


In [ ]:
y_pred_rbf_pka_test = svr_pka.predict(X_pka_test)
y_pred_rbf_hfus_test = svr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_rbf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_rbf_hfus_test)))

Coefficient of determination for test case: 0.37758001944582764
Coefficient of determination for test case: 0.9042218434227742


**Decision Tree**

In [ ]:
dt_regr_pka = DecisionTreeRegressor(max_depth=13,criterion='squared_error',random_state=200)
dt_regr_hfus = DecisionTreeRegressor(max_depth=9,criterion='squared_error',random_state=200)
dt_regr_pka.fit(X_pka_train, y_pka_train)
dt_regr_hfus.fit(X_hfus_train, y_hfus_train)

DecisionTreeRegressor(max_depth=9, random_state=200)

In [ ]:
y_pred_dtse_pka_valid = dt_regr_pka.predict(X_pka_valid)
y_pred_dtse_hfus_valid = dt_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_dtse_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_dtse_hfus_valid)))

Coefficient of determination for validation case: 0.2462267510463254
Coefficient of determination for validation case: 0.44290469335700033


In [ ]:
y_pred_dtse_pka_test = dt_regr_pka.predict(X_pka_test)
y_pred_dtse_hfus_test = dt_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_dtse_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_dtse_hfus_test)))

Coefficient of determination for test case: 0.3075193543939314
Coefficient of determination for test case: 0.8191307654662405


**Ridge Regression**

In [ ]:
ridge_regr_pka = Ridge(alpha=8)
ridge_regr_hfus = Ridge(alpha=8)
ridge_regr_pka.fit(X_pka_train, y_pka_train)
ridge_regr_hfus.fit(X_hfus_train, y_hfus_train)

Ridge(alpha=8)

In [ ]:
y_pred_ridreg_pka_valid = ridge_regr_pka.predict(X_pka_valid)
y_pred_ridreg_hfus_valid = ridge_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_ridreg_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_ridreg_hfus_valid)))

Coefficient of determination for validation case: 0.178481375857945
Coefficient of determination for validation case: 0.5549150398097976


In [ ]:
y_pred_ridreg_pka_test = ridge_regr_pka.predict(X_pka_test)
y_pred_ridreg_hfus_test = ridge_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_ridreg_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_ridreg_hfus_test)))

Coefficient of determination for test case: 0.2509762003630561
Coefficient of determination for test case: 0.895792301612385


**Kernel Ridge-linear**

In [ ]:
kernelridge_linear_regr_pka = KernelRidge(alpha=1.52,kernel='linear')
kernelridge_linear_regr_hfus = KernelRidge(alpha=3.23,kernel='linear')
kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)

KernelRidge(alpha=3.23)

In [ ]:
y_pred_kerreg_linear_pka_valid = kernelridge_linear_regr_pka.predict(X_pka_valid)
y_pred_kerreg_linear_hfus_valid = kernelridge_linear_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_linear_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_linear_hfus_valid)))

Coefficient of determination for validation case: -0.013763264411316367
Coefficient of determination for validation case: 0.6103239367057893


In [ ]:
y_pred_kerreg_linear_pka_test = kernelridge_linear_regr_pka.predict(X_pka_test)
y_pred_kerreg_linear_hfus_test = kernelridge_linear_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_linear_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_linear_hfus_test)))

Coefficient of determination for test case: 0.28220846651342757
Coefficient of determination for test case: 0.8922504777552989


**Kernel Ridge-rbf**

In [ ]:
kernelridge_rbf_regr_pka = KernelRidge(alpha=0.01,kernel='rbf')
kernelridge_rbf_regr_hfus = KernelRidge(alpha=0.01,kernel='rbf')
kernelridge_rbf_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_rbf_regr_hfus.fit(X_hfus_train, y_hfus_train)

KernelRidge(alpha=0.01, kernel='rbf')

In [ ]:
y_pred_kerreg_rbf_pka_valid = kernelridge_rbf_regr_pka.predict(X_pka_valid)
y_pred_kerreg_rbf_hfus_valid = kernelridge_rbf_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_rbf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_rbf_hfus_valid)))

Coefficient of determination for validation case: 0.11273203084042172
Coefficient of determination for validation case: 0.6176468853729609


In [ ]:
y_pred_kerreg_rbf_pka_test = kernelridge_rbf_regr_pka.predict(X_pka_test)
y_pred_kerreg_rbf_hfus_test = kernelridge_rbf_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_rbf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_rbf_hfus_test)))

Coefficient of determination for test case: 0.3196322372230257
Coefficient of determination for test case: 0.8962518682821817


**Kernel Ridge-polynomial**

In [ ]:
kernelridge_poly_regr_pka = KernelRidge(alpha=0.001,kernel='poly',degree=2)
kernelridge_poly_regr_hfus = KernelRidge(alpha=0.001,kernel='poly',degree=2)
kernelridge_poly_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_poly_regr_hfus.fit(X_hfus_train, y_hfus_train)

KernelRidge(alpha=0.001, degree=2, kernel='poly')

In [ ]:
y_pred_kerreg_poly_pka_valid = kernelridge_poly_regr_pka.predict(X_pka_valid)
y_pred_kerreg_poly_hfus_valid = kernelridge_poly_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_poly_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_poly_hfus_valid)))

Coefficient of determination for validation case: -0.29110580293291277
Coefficient of determination for validation case: 0.5296411771336975


In [ ]:
y_pred_kerreg_poly_pka_test = kernelridge_poly_regr_pka.predict(X_pka_test)
y_pred_kerreg_poly_hfus_test = kernelridge_poly_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_poly_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_poly_hfus_test)))

Coefficient of determination for test case: 0.3416261308757603
Coefficient of determination for test case: 0.871332945267997


**Random forests**

In [ ]:
rf_regr_pka = RandomForestRegressor(n_estimators=100,max_depth=18, random_state=200, criterion='squared_error',ccp_alpha=0.0005)
rf_regr_hfus = RandomForestRegressor(n_estimators=100,max_depth=17, random_state=200, criterion='squared_error',ccp_alpha=0.0005)
rf_regr_pka.fit(X_pka_train, y_pka_train)
rf_regr_hfus.fit(X_hfus_train, y_hfus_train)

RandomForestRegressor(ccp_alpha=0.0005, max_depth=17, random_state=200)

In [ ]:
y_pred_rf_pka_valid = rf_regr_pka.predict(X_pka_valid)
y_pred_rf_hfus_valid = rf_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_rf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_rf_hfus_valid)))

Coefficient of determination for validation case: 0.459848698644864
Coefficient of determination for validation case: 0.5713299997725132


In [ ]:
y_pred_rf_pka_test = rf_regr_pka.predict(X_pka_test)
y_pred_rf_hfus_test = rf_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_rf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_rf_hfus_test)))

Coefficient of determination for test case: 0.405974103142293
Coefficient of determination for test case: 0.8885623755142239


**kNN**

In [ ]:
knn_regr_pka = KNeighborsRegressor(n_neighbors=5,weights='uniform',p=2,metric='minkowski')
knn_regr_hfus = KNeighborsRegressor(n_neighbors=7,weights='uniform',p=2,metric='minkowski')
knn_regr_pka.fit(X_pka_train, y_pka_train)
knn_regr_hfus.fit(X_hfus_train, y_hfus_train)

KNeighborsRegressor(n_neighbors=7)

In [ ]:
y_pred_knn_pka_valid = knn_regr_pka.predict(X_pka_valid)
y_pred_knn_poly_hfus_valid = knn_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_knn_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_knn_poly_hfus_valid)))

Coefficient of determination for validation case: 0.3757146573245508
Coefficient of determination for validation case: 0.5086970741017249


In [ ]:
y_pred_knn_pka_test = knn_regr_pka.predict(X_pka_test)
y_pred_knn_hfus_test = knn_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_knn_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_knn_hfus_test)))

Coefficient of determination for test case: 0.27212454805392483
Coefficient of determination for test case: 0.8447012716468906


**Gaussian Process Regression**

In [ ]:
kernel = DotProduct() + WhiteKernel()
gpr_regr_pka = GaussianProcessRegressor(kernel=kernel,random_state=200,n_restarts_optimizer=10,alpha=0.01)
gpr_regr_hfus = GaussianProcessRegressor(kernel=kernel,random_state=200,n_restarts_optimizer=10,alpha=0.01)
gpr_regr_pka.fit(X_pka_train, y_pka_train)
gpr_regr_hfus.fit(X_hfus_train, y_hfus_train)

GaussianProcessRegressor(alpha=0.01,
                         kernel=DotProduct(sigma_0=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, random_state=200)

In [ ]:
y_pred_gpr_pka_valid = gpr_regr_pka.predict(X_pka_valid)
y_pred_gpr_poly_hfus_valid = gpr_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_gpr_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_gpr_poly_hfus_valid)))

Coefficient of determination for validation case: 0.1590844550682068
Coefficient of determination for validation case: 0.5437788551708382


In [ ]:
y_pred_gpr_pka_test = gpr_regr_pka.predict(X_pka_test)
y_pred_gpr_hfus_test = gpr_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_gpr_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_gpr_hfus_test)))

Coefficient of determination for test case: 0.25210758433288805
Coefficient of determination for test case: 0.868679367884972


**Neural Network**

In [ ]:
pka_model = Sequential()
pka_model.add(Dense(200, input_dim=424, activation='relu'))
pka_model.add(Dense(200, input_dim=200, activation='relu'))
pka_model.add(Dense(200, input_dim=200, activation='relu'))
pka_model.add(Dense(1, activation='linear'))

hfus_model = Sequential()
hfus_model.add(Dense(200, input_dim=424, activation='relu'))
hfus_model.add(Dense(200, input_dim=200, activation='relu'))
hfus_model.add(Dense(200, input_dim=200, activation='relu'))
hfus_model.add(Dense(1, activation='linear'))

In [ ]:
tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
pka_model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
hfus_model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])
pka_model.fit(X_pka_train, y_pka_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)
hfus_model.fit(X_hfus_train, y_hfus_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)
 

Epoch 1/50
35/35 [==============================] - 1s 13ms/step - loss: 16.2217 - mean_absolute_percentage_error: 5380283.5000 - val_loss: 9.8651 - val_mean_absolute_percentage_error: 91.8006
Epoch 2/50
35/35 [==============================] - 0s 6ms/step - loss: 9.3292 - mean_absolute_percentage_error: 4758737.5000 - val_loss: 7.6278 - val_mean_absolute_percentage_error: 105.0832
Epoch 3/50
35/35 [==============================] - 0s 6ms/step - loss: 7.6408 - mean_absolute_percentage_error: 3919028.0000 - val_loss: 7.3746 - val_mean_absolute_percentage_error: 115.7636
Epoch 4/50
35/35 [==============================] - 0s 6ms/step - loss: 6.5142 - mean_absolute_percentage_error: 3512550.5000 - val_loss: 6.3085 - val_mean_absolute_percentage_error: 102.6511
Epoch 5/50
35/35 [==============================] - 0s 6ms/step - loss: 5.5041 - mean_absolute_percentage_error: 1548833.3750 - val_loss: 7.6241 - val_mean_absolute_percentage_error: 95.2434
Epoch 6/50
35/35 [======================

In [ ]:
pka_model.fit(X_pka_train, y_pka_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)
hfus_model.fit(X_hfus_train, y_hfus_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)

Epoch 1/50
35/35 [==============================] - 0s 9ms/step - loss: 0.5960 - mean_absolute_percentage_error: 844959.5000 - val_loss: 4.6252 - val_mean_absolute_percentage_error: 42.0424
Epoch 2/50
35/35 [==============================] - 0s 6ms/step - loss: 0.6583 - mean_absolute_percentage_error: 330295.8438 - val_loss: 5.0946 - val_mean_absolute_percentage_error: 37.3941
Epoch 3/50
35/35 [==============================] - 0s 6ms/step - loss: 0.6018 - mean_absolute_percentage_error: 1099345.6250 - val_loss: 4.8376 - val_mean_absolute_percentage_error: 42.9841
Epoch 4/50
35/35 [==============================] - 0s 5ms/step - loss: 0.5759 - mean_absolute_percentage_error: 551417.0000 - val_loss: 6.1804 - val_mean_absolute_percentage_error: 40.4394
Epoch 5/50
35/35 [==============================] - 0s 6ms/step - loss: 0.5711 - mean_absolute_percentage_error: 104737.9531 - val_loss: 4.8627 - val_mean_absolute_percentage_error: 37.0162
Epoch 6/50
35/35 [==============================]

In [ ]:
y_pred_nn_pka_valid = pka_model.predict(X_pka_valid)
y_pred_nn_poly_hfus_valid = hfus_model.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_nn_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_nn_poly_hfus_valid)))

Coefficient of determination for validation case: 0.47450071428130636
Coefficient of determination for validation case: 0.09781101120075941


In [ ]:
y_pred_nn_pka_test = pka_model.predict(X_pka_test)
y_pred_nn_hfus_test = hfus_model.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_nn_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_nn_hfus_test)))

Coefficient of determination for test case: 0.37379748535181945
Coefficient of determination for test case: 0.8897291700170055
